# **INSTALLS AND IMPORTS**

In [2]:
!pip install langchain
!pip install -qU pypdf
!pip install langchain-community
!pip install -qU langchain-pinecone pinecone-notebooks
!pip install -qU langchain-huggingface
!pip install transformers
!pip install huggingface_hub
!pip install pinecone-client
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
from langchain_community.document_loaders import PyPDFLoader
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from huggingface_hub import login

In [ ]:
import pinecone
from huggingface_hub import login
from transformers import pipeline

# **FUNCTIONS**

**Reads a document and splits it into pages, storing them in arrays**.




In [ ]:
def docToPages(file_path):
  loader = PyPDFLoader(file_path)
  docs = []
  docs_lazy = loader.lazy_load()

  for doc in docs_lazy:
      docs.append(doc)

  return docs

# ***VECTORIZATION***

Login and create the Pinecone Key and Index in https://app.pinecone.io/

**Pinecone key**

In [ ]:
pc = Pinecone(api_key="YOUR PINECONE KEY",environment="YOUR ENVIRONMENT")



**Create an index**

Adjust it based on the characteristics of the index you created.

In [ ]:
index_name = "YOUR INDEX"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )


In [ ]:
documents = docToPages('/content/YOUR_PDF.pdf')

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# **PRINT THE TEXT**

In [ ]:
texts = [doc.page_content for doc in documents]
print(texts)

In [ ]:
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)



# **PROMPTS**

In [ ]:
question = input("Ask something")


In [ ]:
results = vector_store.similarity_search(
    question,
    k=2
)

# **OUTPUT**

In [ ]:
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")